### Calibration_Version 2.2.0


This file copies 2.1.1 and does some further experiments. This file changes the moments of the calibration process. We calibrate $\sigma$, $\mu_{c}$ $k$ and $\theta$.

Also we include one more moment into the calibration process: $\Delta D$

The moment of jump-down we are using is the adjusted version.（We only include the left end in the 100 bin regression.)

Changed the last moment: increase in Beta and stayer_mmt

Changed the $\tau_H$ to be 0.225.

Change the weighting matrix: now we minimize using the variance-covariance matrix.

Change the data moments: we use the data moments from our bootstrap process (2000 times). Also the regression moments we use here are from the de-fixed-effect data.

#### version 2.2+ stems from version 2.1.7, where we have got decent and valid estimation results.
We fix bugs in calculating the delta_d here. get_Beta_conditional_expectation

In [1]:
import pandas as pd
import numpy as np
import random
import copy
from matplotlib import pyplot as plt 
from copulas.datasets import sample_trivariate_xyz
from copulas.visualization import scatter_3d, compare_3d
from copulas.multivariate import GaussianMultivariate
from copulas.univariate import BetaUnivariate, GaussianKDE, GaussianUnivariate
from scipy.stats import pareto, beta
from fitter import Fitter, get_common_distributions, get_distributions
from scipy import optimize
from scipy.optimize import fsolve
from scipy.optimize import minimize, rosen, rosen_der
from scipy import stats
import statsmodels.api as sm
from datetime import datetime
%matplotlib inline

In [2]:
path = '/Users/philhuang/Desktop/FDI/workingdata/'

### P1.  Preparation.
#### Load in some functions we need to use for solving for optimal $\eta$ and $\delta$.

In [3]:
# Eqaution 8 where we calculate optimal beta.
def get_optimal_beta(sigma, eta):
    part1     = eta/(1-eta)
    part2     = 1-(sigma-1)*(1-eta)/sigma 
    part3     = 1- (sigma-1)*eta/sigma 
    temp      = np.sqrt(part1*part2/part3)
    beta_star = temp/(1+temp)
    return beta_star

# From (8), we solve for eta given beta. We need to use binary search.
# First, we define a function with which we get the value which is closer to the target.
# when we have two candidates RHS_vals[a]<RHS_vals[b], which means the a_th RHS value smaller than the target 
# and the b_th RHS values bigger than the target.

def get_closer_num(lst,a,b,target):
    baseline = abs(lst[a]-target)
    if abs(lst[b]-target) <= baseline:
        return b
    else:
        return a

# Then we define a binary search function to solve the function.
# In general, lst_a contains the target we want to search for, and we want to return the corresponding 
# element in lst_b
def get_ans(lst_a, lst_b, target):
    
    left=0 
    right=len(lst_a)
    
    while left<right:
        middle = left+ ((right - left)//2)
        if lst_a[middle] == target:
            return lst_b[middle]
        if lst_a[middle] < target:
            if lst_a[middle+1] >= target:
                idx_g = get_closer_num(lst_a, middle, middle+1, target)
                return lst_b[idx_g]
            else:
                left = middle+1
        if lst_a[middle] > target:
            if lst_a[middle-1] <= target:
                idx_g = get_closer_num(lst_a, middle-1, middle, target)
                return lst_b[idx_g]
            else:
                right = middle

# Equation 7. Given sigma, eta and beta, we get delta. 
# Note that since beta* is a function of sigma and eta, delta is a function of sigma and eta too. 
def get_delta(sigma, eta, beta):
    part1 = sigma - (sigma-1)*(beta*eta+(1-beta)*(1-eta))
    part2 = (((1/beta)**eta) * ((1/(1-beta))**(1-eta)))**(1-sigma)
    delta = part1*part2
    return delta


### P3. Simulate the bunching pattern and see how the firm-level outcomes would be like.

The firm-level outcomes can be expressed with:
\begin{gather}
    \frac{\tilde{h}}{\tilde{m}} = \frac{w_{m} \beta \eta}{w_{h} (1-\beta) (1-\eta)}  \tag{15}\\ 
    \tilde{p}(\varphi) = \frac{\sigma}{\sigma-1}
                         \left(\frac{w_{h}}{\beta \eta}\right)^{\eta}
                         \left(\frac{w_{m}}{(1-\beta)(1-\eta)}\right)^{1-\eta}
                         \frac{1}{\varphi}    \tag{17}\\
    \tilde{x}(\varphi) = \tilde{p}(\varphi)^{1-\sigma}P^{\sigma-1}X(1-\tau)^{\sigma} \tag{16} \\
    \tilde{h}   = \frac{\sigma-1}{\sigma} \frac{\beta \eta}{w_{h}}\tilde{x}(\varphi)  \tag{19} \\
    \tilde{m}   = \frac{\sigma-1}{\sigma} \frac{(1-\beta)(1- \eta)}{w_{m}}\tilde{x}(\varphi)  \tag{18} \\
    \tilde{\pi} = \left[ 1-\frac{\sigma-1}{\sigma}(\beta \eta +(1-\beta)(1-\eta)) \right] \tilde{x} \tag{*}
\end{gather}

With equation (6), we get $\delta$ :
\begin{equation}
    \delta(\sigma, \eta, \beta) = [\sigma-(\sigma-1)(\beta\eta+(1-\beta)(1-\eta))]
     [(\frac{1}{\beta})^{\eta} (\frac{1}{1-\beta})^{1-\eta}]^{1-\sigma}  \qquad(6)
\end{equation}

Then we use equation (7). Here we need to see whether:

\begin{equation}
    \delta(\sigma, \eta, \beta^{*}) \geq (\delta(\sigma, \eta, \underline{\beta})-c) [\frac{1-\tau^{L}}{1-\tau^{H}}]^{\sigma}  \qquad (7)
\end{equation}

Note that:
\begin{equation}
    p^{*}(\varphi) = \frac{\sigma}{\sigma-1} (\frac{w_{h}}{\eta})^{\eta}  (\frac{w_{m}}{1-\eta})^{1-\eta} \frac{1}{\varphi}
\end{equation}

In [4]:
# Equation 7. Given sigma, eta and beta, we get delta. 
# Note that since beta* is a function of sigma and eta, delta is a function of sigma and eta too. 
def get_delta(sigma, eta, beta):
    part1 = sigma - (sigma-1)*(beta*eta+(1-beta)*(1-eta))
    part2 = (((1/beta)**eta) * ((1/(1-beta))**(1-eta)))**(1-sigma)
    delta = part1*part2
    return delta

# Equation (7). See if a firm would choose to bunch or not. 
def bunch_or_not(sigma, eta, beta_optimal, c, tau_L, tau_H):
    
    # Calculate the rhs of equation 9. 
    part1 = get_delta(sigma, eta, 0.25)-c
    part2 = ((1-tau_L)/(1-tau_H))**sigma
    RHS   = part1*part2
    
    # Calculate the LHS of equation (7). 
    LHS   = get_delta(sigma, eta, beta_optimal)
    return int(LHS>=RHS)  # here 0 means it is optimal to bunch, 1 means to stay and not to bunch.

# Input equation (15-19)
def get_FirmVar(beta, eta, sigma, wh, wm, tau, varphi, P, X, bunch_or_not, cost):
    
    p_star  = (sigma/(sigma-1))* ((wh/eta)**eta) * ((wm/(1-eta))**(1-eta)) /varphi
    p_tild  = (sigma/(sigma-1)) * (wh/(beta*eta))**eta * (wm/((1-beta)*(1-eta)))**(1-eta) / varphi
    x_tild  = p_tild**(1-sigma) * P**(sigma-1) * X * (1-tau)**sigma
    h_tild  = ((sigma-1)/sigma) * (beta*eta/wh) * x_tild
    m_tild  = ((sigma-1)/sigma) * ((1-beta)*(1-eta)/wm) * x_tild
    
    # Note that if ther is no bunching (bunch_or_not == 1), the final Pi is just Pi_tild.
    # If there is bunching (bunch_or_not == 0), then the final Pi should deduct the cost.
    pi_tild = (get_delta(sigma, eta, beta)-cost*(1-bunch_or_not))* p_star**(1-sigma) * P**(sigma-1)* X*((1-tau)**sigma)/sigma

    #return p_tild, x_tild, h_tild, m_tild, pi_tild
    return pi_tild


# Now we define a function to calculate optimal eta for these firms.

def get_eta(beta, sigma):
    # With beta_bins, calculate the LHS of (8). This is the list of targets we are searching for. 
    LHS_8 = np.array(beta)/(1-np.array(beta))

    # Define the space of eta, which is the lst_b in the get_ans function. 
    eta_space = np.linspace(0.0001, 0.9999, 1000000)

    # Calculate the RHS of equation 8, which is the lst_a in the get_ans function. 
    part1  = eta_space/(1-eta_space)
    part2  = 1-(sigma-1)*(1-eta_space)/sigma 
    part3  = 1- (sigma-1)*eta_space/sigma 
    RHS_8  = np.sqrt(part1*part2/part3)

    eta_simul = np.zeros(len(beta))

    for i in range(len(beta)): 
        eta_simul[i] = get_ans(RHS_8, eta_space, LHS_8[i])
    
    return eta_simul

# We define a function to generate cost and productivity.
def getCostVarphi(theta, mu_c, k, Rep_id):
    # Generate adjustment cost and productivity.
    # See the process of drawing these random numbers in 
    # section B.1 Online Appendix of James & Ariell (2015 CJE)
    np.random.seed(1234+Rep_id)
    copula_u = np.random.uniform(0,1,101310)

    np.random.seed(123+Rep_id)
    copula_x = np.random.uniform(0,1,101310)

    copula_a = copula_x*(1-copula_x)
    copula_b = theta + copula_a*((theta-1)**2)
    copula_c = 2*copula_a*(copula_u*(theta**2)+1-copula_u) + theta*(1-2*copula_a)
    copula_d = np.sqrt(theta)*np.sqrt(theta + 4*copula_a*copula_u*(1-copula_u)*((1-theta)**2))
    copula_v = (copula_c-(1-2*copula_x)*copula_d)/(2*copula_b)

    # We assume that cost is just a uniform distribution.
    copula_cost = copula_u*mu_c

    # Now we generate varphi based on U we draw.
    # If U is drawn from a uniform distribution [0,1], and T = xm*U**(-1/k), then T is Pareto-Distributed.
    copula_varphi = xm*((1-copula_v)**(-1/k))

    cost_varphi_df = pd.DataFrame(data={'copula_cost':copula_cost, 'copula_varphi':copula_varphi})

    # Winsorize the sample. Drop the bottom 2% and top 1% sample.
    top99   = np.quantile(copula_varphi, 0.99)
    bottom1 = np.quantile(copula_varphi, 0.01)
    eta_varphi_winsorized = copy.deepcopy(cost_varphi_df[(cost_varphi_df['copula_varphi']<top99)&(cost_varphi_df['copula_varphi']>bottom1)])
    eta_varphi_winsorized.reset_index(drop=True, inplace=True)
    cost_simul   = eta_varphi_winsorized['copula_cost'].values[:a1]
    varphi_simul = eta_varphi_winsorized['copula_varphi'].values[:a1]

    return cost_simul, varphi_simul


#### Read in some parameters and $\beta$

In [5]:
# We read in bins (beta), eta data and see their distributions. 
beta_eta_df   = pd.read_csv(path+'beta_eta.csv')
beta_all      = np.array(beta_eta_df['beta_bins'])
#eta_all      = np.array(beta_eta_df['eta_bins'])

# Read in bins, real distribution and counterfactual distributions of firms we use in the bunching estimation.
bunching_df   = pd.read_excel(path+'R2_pre2008June122023.xlsx')

bins_bunching = np.array(bunching_df['R2_bins'])
C_bunching    = np.array(bunching_df['R2_C_all'])  # Actual probability of firms falling into each bins.
h_bunching    = np.array(bunching_df['h'])         # Counterfactual probability of firms falling into each bins.
p             = 4                                  # Polynomial.
minD          = 0.01                               # Minimum of the bins we use.

tau_L         = np.float64(0.15)
tau_H         = np.float64(0.225)
w_h           = 1
w_m           = 1
big_p         = 1
big_x         = 1
xm            = 1

In [6]:
# Function that helps us to calculate the conditional expectation of Beta.
def get_Beta_conditional_expectation(group_name, count_var, target_df):
    
    # Calculate the number of firms in each bin.
    temp_counts = target_df.groupby(group_name)[count_var].count().reset_index()
    temp_counts.rename(columns={count_var:'counts'}, inplace=True)
    
    # Calculate the probability of firms falling into each bin.
    temp_counts['probs'] = temp_counts['counts']/np.sum(temp_counts['counts'])
    
    # Calculate the conditional probability of firms with FDI share below 25%.
    temp_counts = copy.deepcopy(temp_counts[(temp_counts[group_name]<=25)&(temp_counts[group_name]>=7)])
    temp_counts['conditional_prob'] = temp_counts['probs']/np.sum(temp_counts['probs'])
    
    # Calculate conditional expectation.
    expetation = np.sum((temp_counts[group_name]+1)*temp_counts['conditional_prob']/100)
    
    # Calculate frac_nb (staying ratio)
    temp_counts = copy.deepcopy(temp_counts[(temp_counts[group_name]<25)&(temp_counts[group_name]>=7)])
    sum_probs   = np.sum(temp_counts['probs'])
    # sum_probs   = np.sum(temp_counts['probs'][:-1])

    
    return expetation, sum_probs

#### Construct data moments and weighting matrix.

In [7]:
####################################################################################################
# mean and covariance of the jump-down and the profit dispersion. 
# First read in the moment for the drop at the notch.  
####################################################################################################

rst_bootstrap100bins = pd.read_csv(path+'rst_bs100binDeFEbsample.csv')

# Take out the estimated parameters of the bin-effect that we care about.
bootstrapb_fdi_24    = rst_bootstrap100bins['b_fdi_24'].values
bootstrapb_fdi_25    = rst_bootstrap100bins['b_fdi_25'].values
bootstrapb_fdi_26    = rst_bootstrap100bins['b_fdi_26'].values
bootstrapb_fdi_27    = rst_bootstrap100bins['b_fdi_27'].values

# Read in the firm distribution data. 
rst_bootstrapBinDistribution = pd.read_csv(path+'rst_bootstrapBinDistribution.csv')
bootstrap_counts_25          = rst_bootstrapBinDistribution['counts_25'].values
bootstrap_counts_26          = rst_bootstrapBinDistribution['counts_26'].values
# Calculate the weight.
bootstrap_weight_25          = bootstrap_counts_25/(bootstrap_counts_25+bootstrap_counts_26)
bootstrap_weight_26          = bootstrap_counts_26/(bootstrap_counts_25+bootstrap_counts_26)

# Calculate the weighted sum of the jump at the notch.
weighted_drop     = bootstrapb_fdi_25*bootstrap_weight_25 + bootstrapb_fdi_26*bootstrap_weight_26

# Calculate the bootstrapped profit drop at the notch.
left_drop_BS  = bootstrapb_fdi_24-weighted_drop
right_drop_BS = bootstrapb_fdi_27-weighted_drop

################################################################################
# Calculate the dispersion of the profits in the data.
################################################################################
# Read in the bootstrapped results.
rst_bootstrap20bins = pd.read_csv(path+'rst_bs20binDeFEbsample.csv')
bootstrapb_fdi_1    = rst_bootstrap20bins['b_fdi_1'].values
bootstrapb_fdi_9    = rst_bootstrap20bins['b_fdi_9'].values
profitDis_BS        = bootstrapb_fdi_1-bootstrapb_fdi_9

################################################################################
# Calculate the tfp moments.
################################################################################
# We use the ratio of 25 percentile/50 percentile, and 75 percentile/50 percentile.
rst_bootstrapTFP = pd.read_csv(path+'rst_bootstrapTFP.csv')
tfp_p_25         = rst_bootstrapTFP['p_25'].values
tfp_p_50         = rst_bootstrapTFP['p_50'].values
tfp_p_75         = rst_bootstrapTFP['p_75'].values

# Calculate the ratio.
lntfp_2550_BS    = tfp_p_25/tfp_p_50
lntfp_7550_BS    = tfp_p_75/tfp_p_50

################################################################################
# Moment of the stayer ratio and increased FDI share.
################################################################################
bootstrapBucnhing = pd.read_csv(path+'bootstrap_bunching12June2023.csv')
frac_nb_BS        = bootstrapBucnhing['frac_nb_B'].values
Dd_BS             = bootstrapBucnhing['Dd_B'].values

################################################################################
# Collect the moments and construct variance-covariance matrix.
################################################################################
# Put all bootstrapped data together. 
BS_results      = np.zeros((2000, 7))
BS_results[:,0] = left_drop_BS
BS_results[:,1] = right_drop_BS
BS_results[:,2] = profitDis_BS

BS_results[:,3] = lntfp_2550_BS
BS_results[:,4] = lntfp_7550_BS

BS_results[:,5] = frac_nb_BS
BS_results[:,6] = Dd_BS

# get data moments
data_moments = np.around(BS_results.mean(axis=0),4)

# calculate the var matrix.
var_matrix   = np.zeros((7,7))
cov_regress  = np.array(pd.DataFrame(BS_results[:,:3]).cov())
cov_lntfp    = np.array(pd.DataFrame(BS_results[:,3:5]).cov())
cov_bunching = np.array(pd.DataFrame(BS_results[:,5:7]).cov())

# Store those matrix into the var_matrix.
var_matrix[:3, :3]   = cov_regress
var_matrix[3:5, 3:5] = cov_lntfp
var_matrix[5:7, 5:7] = cov_bunching

In [8]:
pd.DataFrame(var_matrix)

,0,1,2,3,4,5,6
0,0.008581,0.000400,-0.000014,0.000000e+00,0.000000e+00,0.000000,0.000000
1,0.000400,0.005671,0.000114,0.000000e+00,0.000000e+00,0.000000,0.000000
2,-0.000014,0.000114,0.005088,0.000000e+00,0.000000e+00,0.000000,0.000000
3,0.000000,0.000000,0.000000,2.000168e-07,-9.526837e-09,0.000000,0.000000
4,0.000000,0.000000,0.000000,-9.526837e-09,2.528222e-07,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.007888,-0.001867
6,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,-0.001867,0.001202


### Calculate the simulated moments.

#### We first simulate 100,000 $\beta$. Note that $\beta$ does not change with the parameters in the following part. 

In [9]:
# Read in b that we estimated from the bunching part.
b = np.array(pd.read_excel(path+'outputJune122023/R5_pre2008June122023.xlsx')['b'])

# Now we want to get the distribution of eta, and try to fit it to a beta-distribution. 
# In order to do so, we omit all the round number effects. 
polynomial_beta = []
for i in range(p+1):
    polynomial_beta.append(bins_bunching**i)
polynomial_beta = np.array(polynomial_beta).T

# Multiply b and polynomial_beta together.
distribution_noRN = np.dot(polynomial_beta, b[:p+1].reshape(p+1,-1))
distribution_noRN = distribution_noRN.ravel()

# Now we simulate 100,000 firms. 
# The first step is to generate 100,000 betas according to the counterfactual distribution of Beta.
# Now we know the probability of etas falling into each bins, we simulate data of eta based on this.
# cf means counterfactual here.
beta_cf_simul = []

# We cut the last bin (0.99-1) out since there would be a lot of extrem values in that bin when we calculate eta.
bins_bunching = np.array(bunching_df['R2_bins'])

# We get lower and uppper bond of each bin, and sample beta within each bin.
beta_temp  = list(bins_bunching)
beta_temp.append(np.float64(1.0))
total_firm_num = 100000

for i in range(len(beta_temp)-1):
    lb_bin  = beta_temp[i]     # lower bond of the bin
    ub_bin  = beta_temp[i+1]   # upper bond of the bin 
    len_bin = ub_bin-lb_bin
    
    # Number of firms in each bin, each should have an beta that lies within the range of the bin. 
    # Here we assume that in each bin, beta is uniformly distributed.
    bin_firm_num = int(h_bunching[i]*total_firm_num)
    np.random.seed(123+i)
    
    # Random.random_sample generates uniformly-distributed sample in [0,1). So here we need to multiply them
    # by the length of each bin, and add lb_bin to it. 
    rand_uniform   = np.random.random_sample(size = bin_firm_num)
    beta_each_bin  = list(rand_uniform*len_bin + lb_bin)
    beta_cf_simul += beta_each_bin

# Now we put all the simulated_beta_star into each bins.
bins_temp = np.arange(2,101)/100

# Here the ID of the bins starts from 1 (not 0). So the first bin [0.02,0.03) has a simul_bin_id==1.
simul_bin_id = np.digitize(beta_cf_simul, bins_temp, right=False)

# Count how many betas are there in each bin.
simul_beta_df      = pd.DataFrame(data={'beta_cf_simul':np.array(beta_cf_simul), 
                                        'simul_bin_id':np.array(simul_bin_id)+1})
simul_betaCount_df = simul_beta_df.groupby('simul_bin_id')['beta_cf_simul'].count().reset_index()
simul_betaCount_df.rename(columns={'beta_cf_simul':'count'}, inplace=True)

# Count the number of firms that may choose to bunch (FDI_SAHRE<25)
num_possible_bunchFirm = np.sum(simul_betaCount_df[simul_betaCount_df['simul_bin_id']<25]['count'])

# Let's check how our simulated beta looks like. 
a1 = len(beta_cf_simul)
a2 = int(np.sum(h_bunching)*total_firm_num)
print('Number of simulated firms: {0}'.format(a1))
print('Number of counterfactual firms: {0}'.format(a2))
print('Note that there should be less than 100,000 firms since we exclude firms with fdi_share>0.99, and fdi_share less than 0.02.')
print('There is some slight difference between the number of counterfactual and simulated firms because we round the floats to integers.')



Number of simulated firms: 99277
Number of counterfactual firms: 99321
Note that there should be less than 100,000 firms since we exclude firms with fdi_share>0.99, and fdi_share less than 0.02.
There is some slight difference between the number of counterfactual and simulated firms because we round the floats to integers.


In [10]:
# Define a function to calculate the simulated moments.
def getSimulMMt(sigma, theta, k, mu_c, Reps):
    # Store the results.
    simulated_mmtRst = np.zeros(7)
    
    # Generate eta, given optimal beta.
    # Note that when beta>0.99, there would be a lot of extreme values when we calculate eta.
    # For simplicity, if firms have betas>0.99, we replace them to be 0.99.
    simul_beta_df.loc[simul_beta_df['beta_cf_simul']>0.99, ['beta_cf_simul']]=0.99
    eta_simul = get_eta(simul_beta_df['beta_cf_simul'].values, sigma)
    simul_beta_df['eta_simul'] = np.array(eta_simul)

    # Generate simulated cost and varphi.
    for Rep in range(Reps):
        cost_simul, varphi_simul = getCostVarphi(theta, mu_c, k, Rep)
        
        # Pack simulated data into a dataFrame.
        sim_firm_df = pd.DataFrame(data={'beta_cf_simul': simul_beta_df['beta_cf_simul'],
                                         'simul_bin_id': simul_beta_df['simul_bin_id'],
                                         'eta_simul': simul_beta_df['eta_simul'],
                                         'cost_simul':cost_simul,
                                         'varphi_simul': varphi_simul})

        # Calculate the pre-tax profits without bunching.
        # Start by giving all firms a low tax rate.
        sim_firm_df['tau_noBunch'] = tau_L
        # If the FDI share is below 0.25, replace the tax rate with a higher rate.
        sim_firm_df.loc[sim_firm_df['beta_cf_simul']<0.25, ['tau_noBunch']] = tau_H

        # Calculate post-tax profits when there is no bunching.
        pi_noBunchpostTax = get_FirmVar(sim_firm_df['beta_cf_simul'], sim_firm_df['eta_simul'], 
                                        sigma, w_h, w_m, 
                                        sim_firm_df['tau_noBunch'], 
                                        sim_firm_df['varphi_simul'], 
                                        big_p, big_x, 1, 0)
        
        # Pre-tax profits when there is no bunching.
        sim_firm_df['pi_noBunch'] = np.log(pi_noBunchpostTax/(1-sim_firm_df['tau_noBunch']))

        # Now we look at firms bunching decisions. We first calculate the LHS and RHS of 7
        # Calculate the rhs of equation 7 

        part1 = get_delta(sigma, sim_firm_df['eta_simul'], 0.25)-sim_firm_df['cost_simul']
        part2 = ((1-tau_L)/(1-tau_H))**sigma
        RHS   = part1*part2

        # Calculate the LHS of equation (7). 
        LHS   = get_delta(sigma, sim_firm_df['eta_simul'], sim_firm_df['beta_cf_simul'])
        bunch_decision = (LHS>=RHS).astype(int) # here 0 means it is optimal to bunch, 1 means to stay and not to bunch.

        # Compile bunching decision into the dataframe.
        sim_firm_df['bunch_decision'] = bunch_decision

        # When sim_firm_df['beta_cf_simul']>=0.25, there is no bunching.
        sim_firm_df.loc[sim_firm_df['beta_cf_simul']>=0.25, ['bunch_decision']]=1

        # Let's create a column of firms' final beta: when there is no bunching, final_beta = beta_cf_simul;
        # When there is bunching, final_beta = 0.25.
        sim_firm_df['final_beta'] = sim_firm_df['beta_cf_simul']
        sim_firm_df.loc[sim_firm_df['bunch_decision']==0, ['final_beta']] = 0.25

        # Calculate final_bin_ID.
        # Here the ID of the bins starts from 1 (not 0). So the first bin [0.02,0.03) has a simul_bin_id==1.
        sim_firm_df['final_bin_id'] = np.digitize(sim_firm_df['final_beta'], bins_temp, right=False)
        sim_firm_df['final_bin_id'] = sim_firm_df['final_bin_id']+1

        # Note that eta does not change, so final_eta = eta_simul.
        sim_firm_df['final_eta'] = sim_firm_df['eta_simul']

        # Generate final_tau column. If final_beta>=0.25, then final_tau=tau_L. If final_beta<0.25, final_tau= tau_H.
        sim_firm_df['final_tau'] = tau_H
        sim_firm_df.loc[sim_firm_df['final_beta']>=0.25, ['final_tau']]=tau_L

        # Calculate simulated profit before tax, with bunching. 
        pi_tild = get_FirmVar(sim_firm_df['final_beta'],sim_firm_df['final_eta'],
                              sigma, w_h, w_m, 
                              sim_firm_df['final_tau'],sim_firm_df['varphi_simul'],
                              big_p, big_x, 
                              sim_firm_df['bunch_decision'],sim_firm_df['cost_simul'],)
        
        # Calculate profits before tax with bunching.
        sim_firm_df['pi_preTax'] = np.log(pi_tild/(1-sim_firm_df['final_tau']))

        # Calculate the mean of pi in each bins and de-mean.
        sub_mean_pi = pd.DataFrame(sim_firm_df.groupby('final_bin_id')['pi_preTax'].mean()).reset_index()

        # Take out the bins 0.24, 0.25,0.26.
        profit_bin2426 = sub_mean_pi[(sub_mean_pi['final_bin_id']>=24)&(sub_mean_pi['final_bin_id']<=26)]['pi_preTax'].values

        # Calculate the jump. Note that we also need the profit in 0.24 to be higher than the profit in 0.26.
        left_drop  = profit_bin2426[0] - profit_bin2426[1]
        right_drop = profit_bin2426[2] - profit_bin2426[1]
        over_drop  = profit_bin2426[0] - profit_bin2426[2]
        #simulated_drop_moment = [left_drop, right_drop, over_drop]
        simulated_drop_moment = [left_drop, right_drop]

        ### Calculate the increase in Beta and stayer_mmt.
        notch_bin = 25
        minBin    = 1
        
        # Calculate the counterfactual distribution of firms (corresponding to h in the bunching estimation).
        temp_h      = sim_firm_df.groupby('simul_bin_id')['beta_cf_simul'].count().reset_index()
        temp_h['h'] = temp_h['beta_cf_simul']/np.sum(temp_h['beta_cf_simul'])

        # Calculate the 'observed' distribution of firms (corresponding to C in the bunching estimation).
        # Note that here, the 'observed' distribution is the distribution of firms with bunching.
        temp_C      = sim_firm_df.groupby('final_bin_id')['beta_cf_simul'].count().reset_index()
        temp_C['C'] = temp_C['beta_cf_simul']/np.sum(temp_C['beta_cf_simul'])
        
        # Theoretically, all firms should have bunched to the right. However due to firction cost, 
        # some firms still stay to the left. Here we calculate the ratio of real stayers over conterfactual stayers. 
        # The higher the ratio, the higher the frition cost 
        frac_nb = np.sum(temp_C[temp_C['final_bin_id']<notch_bin]['C'])/np.sum(temp_h[temp_h['simul_bin_id']<notch_bin]['h'])

        # b_Bstat: the upper bound of bins of the range (minD, notch].
        b_Bstat = np.arange(minBin+1, notch_bin+1)/100+0.01

        # c_Bstat: conditional probability of "observed" firm distribution
        c_Bstat = np.array(temp_C[temp_C['final_bin_id']<= notch_bin]['C'])/np.sum(temp_C[temp_C['final_bin_id']<= notch_bin]['C'])

        # hh: conditional probability of "counter-factual" firm distribution
        hh = np.array(temp_h[temp_h['simul_bin_id']<= notch_bin]['h'])/np.sum(temp_h[temp_h['simul_bin_id']<= notch_bin]['h'])

        # Calculate Dd
        Dd = np.sum(b_Bstat*(c_Bstat - hh))/np.sum(b_Bstat*hh)
        
        stayer_mmt    = [frac_nb]
        beta_increase = [Dd]

        ### Calculate the simulated TFP moment.
        lnsimul_TFP          = np.log(varphi_simul)
        lnsimul_TFP_2550     = np.percentile(lnsimul_TFP, 0.25)/np.percentile(lnsimul_TFP, 0.5)
        lnsimul_TFP_7550     = np.percentile(lnsimul_TFP, 0.75)/np.percentile(lnsimul_TFP, 0.5)
        simulated_TFP_moment = [lnsimul_TFP_2550, lnsimul_TFP_7550]  
        
        # Calculate the profit dispersion.
        pi_preTaxBin1       = np.mean(sim_firm_df[sim_firm_df['final_bin_id']<5]['pi_preTax'])
        pi_preTaxBin50      = np.mean(sim_firm_df[(sim_firm_df['final_bin_id']<50)&
                                                  (sim_firm_df['final_bin_id']>=45)]['pi_preTax'])
        simul_pi_dispersion = [pi_preTaxBin1-pi_preTaxBin50]

        simulated_mmt = np.array(simulated_drop_moment+ simul_pi_dispersion + simulated_TFP_moment + stayer_mmt
                                 + beta_increase )
        simulated_mmtRst += simulated_mmt
        
        
    return simulated_mmtRst/Reps


In [14]:
# Test our code.
simulated_mmtRst = getSimulMMt(2.2, 20, 2.1, 0.35, 10)
print(simulated_mmtRst)
print(data_moments)

[0.26208171 0.06304872 1.2047081  0.83280396 1.16251036 0.59697406
 0.19901692]
[0.2228 0.0647 1.3992 0.83   1.16   0.4609 0.2082]


In [15]:
# Calculate the distance between simulated conditions and data conditions.
# We first specify the weighting matrix.
w = np.linalg.inv(var_matrix)

# Define the objective function.
def get_MomentCondition(Paras, Reps):
    sigma    = Paras[0]
    theta    = Paras[1]
    k        = Paras[2]
    mu_c     = Paras[3]
    SimulMMt = getSimulMMt(sigma, theta, k, mu_c, Reps)
    dif      = SimulMMt-data_moments
    distance = np.matmul(np.matmul(dif, w), dif.T)
    return float(distance)

In [16]:
params_ini = [2.2, 20, 2.1, 0.35]
Reps       = 5

Nfeval = 1

def callbackF(xk):
    global Nfeval
    print ('{0:4d} {1: 3.6f}  {2: 3.6f}  {3: 3.6f}  {4: 3.6f}  {5: 3.6f}'\
           .format(Nfeval, get_MomentCondition(xk, Reps), xk[0], xk[1], xk[2], xk[3]))
    Nfeval += 1

print  ('{0:4s}     {1:9s}   {2:9s}  {3:9s}  {4:9s}  {5:9s}'\
        .format('Iter', 'f(X)',  ' sigma', ' theta',  'k', 'mu_c'))

start_time = datetime.now()
betas = minimize(get_MomentCondition, x0=params_ini, 
                 args=(Reps), 
                 method='Nelder-Mead', 
                 tol=0.0001,
                 callback = callbackF, 
                 options={'maxiter':1000})

final_sigma1e51, final_theta1e51, final_k1e51, final_muc1e51 = betas['x']

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Iter     f(X)         sigma      theta     k          mu_c     
   1  50.782136   2.310000   20.000000   2.100000   0.350000
   2  50.782136   2.310000   20.000000   2.100000   0.350000
   3  50.782136   2.310000   20.000000   2.100000   0.350000
   4  50.782136   2.310000   20.000000   2.100000   0.350000
   5  43.031994   2.267031   20.062500   2.111484   0.351914
   6  39.731062   2.305273   20.046875   2.147988   0.333936
   7  39.452673   2.313652   20.054688   2.077236   0.342925
   8  39.452673   2.313652   20.054688   2.077236   0.342925
   9  39.452673   2.313652   20.054688   2.077236   0.342925
  10  39.452673   2.313652   20.054688   2.077236   0.342925
  11  39.452673   2.313652   20.054688   2.077236   0.342925
  12  39.452673   2.313652   20.054688   2.077236   0.342925
  13  38.874767   2.329773   20.051783   2.113930   0.333926
  14  38.874767   2.329773   20.051783   2.113930   0.333926
  15  38.874767   2.329773   20.051783   2.113930   0.333926
  16  38.874767   2.3

In [17]:
# Test our code.
simulated_mmtRst = getSimulMMt(final_sigma1e51, final_theta1e51, final_k1e51, final_muc1e51, 10)
print(simulated_mmtRst)
print(data_moments)

[0.29651663 0.04156448 1.37693757 0.83285678 1.16225029 0.54995858
 0.22144749]
[0.2228 0.0647 1.3992 0.83   1.16   0.4609 0.2082]


In [18]:
params_ini = [2.2, 17.8, 2.3, 0.3]
Reps       = 5

Nfeval = 1

def callbackF(xk):
    global Nfeval
    print ('{0:4d} {1: 3.6f}  {2: 3.6f}  {3: 3.6f}  {4: 3.6f}  {5: 3.6f}'\
           .format(Nfeval, get_MomentCondition(xk, Reps), xk[0], xk[1], xk[2], xk[3]))
    Nfeval += 1

print  ('{0:4s}     {1:9s}   {2:9s}  {3:9s}  {4:9s}  {5:9s}'\
        .format('Iter', 'f(X)',  ' sigma', ' theta',  'k', 'mu_c'))

start_time = datetime.now()
betas = minimize(get_MomentCondition, x0=params_ini, 
                 args=(Reps), 
                 method='Nelder-Mead', 
                 tol=0.0001,
                 callback = callbackF, 
                 options={'maxiter':1000})

final_sigma1e52, final_theta1e52, final_k1e52, final_muc1e52 = betas['x']

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Iter     f(X)         sigma      theta     k          mu_c     
   1  10.198900   2.310000   17.800000   2.300000   0.300000
   2  10.198900   2.310000   17.800000   2.300000   0.300000
   3  10.198900   2.310000   17.800000   2.300000   0.300000
   4  7.710956   2.300547   17.883437   2.088867   0.313711
   5  7.710956   2.300547   17.883437   2.088867   0.313711
   6  7.710956   2.300547   17.883437   2.088867   0.313711
   7  7.710956   2.300547   17.883437   2.088867   0.313711
   8  7.710956   2.300547   17.883437   2.088867   0.313711
   9  7.710956   2.300547   17.883437   2.088867   0.313711
  10  7.710956   2.300547   17.883437   2.088867   0.313711
  11  7.710956   2.300547   17.883437   2.088867   0.313711
  12  7.710956   2.300547   17.883437   2.088867   0.313711
  13  7.573437   2.311439   17.877239   2.133430   0.306403
  14  7.445378   2.312507   17.881117   2.076212   0.310466
  15  7.445378   2.312507   17.881117   2.076212   0.310466
  16  7.445378   2.312507   17.88

 137  6.335332   2.387614   17.880655   2.273812   0.335324
 138  6.335303   2.387727   17.880654   2.274250   0.335340
 139  6.335303   2.387727   17.880654   2.274250   0.335340
 140  6.335303   2.387727   17.880654   2.274250   0.335340
 141  6.335273   2.387826   17.880654   2.274588   0.335345
 142  6.335273   2.387826   17.880654   2.274588   0.335345
 143  6.335273   2.387826   17.880654   2.274588   0.335345
 144  6.335273   2.387826   17.880654   2.274588   0.335345
 145  6.335262   2.387841   17.880655   2.274765   0.335349
 146  6.335260   2.387841   17.880654   2.274691   0.335348
 147  6.335253   2.387850   17.880654   2.274762   0.335349
 148  6.335253   2.387850   17.880654   2.274762   0.335349
 149  6.335253   2.387850   17.880654   2.274762   0.335349
 150  6.335253   2.387850   17.880654   2.274762   0.335349
 151  6.335253   2.387850   17.880654   2.274762   0.335349
 152  6.335253   2.387850   17.880654   2.274762   0.335349
Duration: 0:13:03.585016


In [19]:
# Test our code.
simulated_mmtRst = getSimulMMt(final_sigma1e52, final_theta1e52, final_k1e52, final_muc1e52, 10)
print(simulated_mmtRst)
print(data_moments)

[0.24715408 0.0245752  1.39566964 0.83316151 1.16178603 0.56783955
 0.21245643]
[0.2228 0.0647 1.3992 0.83   1.16   0.4609 0.2082]


In [20]:
calibration_rst1 = np.array([final_sigma1e51, final_theta1e51, final_k1e51, final_muc1e51])
calibration_rst2 = np.array([final_sigma1e52, final_theta1e52, final_k1e52, final_muc1e52])
calibration_rst  = np.stack((calibration_rst1, calibration_rst2))
calibration_rst

array([[ 2.34616035, 20.05298081,  2.15190703,  0.32033915],
       [ 2.38784976, 17.88065423,  2.27476246,  0.33534905]])

In [21]:
calibration_rst = pd.DataFrame(calibration_rst, columns=['sigma', 'theta', 'k', 'mu_c'])
calibration_rst

,sigma,theta,k,mu_c
0,2.34616,20.052981,2.151907,0.320339
1,2.38785,17.880654,2.274762,0.335349


In [22]:
calibration_rst.to_csv(path +'calibration_rstJune122023.csv', index=False)

In [24]:
## Compute the Jacobian matrix (rows: moments, columns: parameters)

theta = np.array([final_sigma1e51, final_theta1e51, final_k1e51, final_muc1e51])

# Note that we have 7 moments and 4 parameters.
Nmom  = 7
nparm = 4
G     = np.zeros((Nmom,nparm))

# around 5% pertubation(theta*0.05)
h = theta*0.05
h

array([0.11730802, 1.00264904, 0.10759535, 0.01601696])

In [25]:
G

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [26]:
for i in range(nparm):
    thetah    = copy.deepcopy(theta)
    thetah[i] = thetah[i]+h[i]
    thetal    = copy.deepcopy(theta)
    thetal[i] = thetal[i]-h[i]
    momentsh  = getSimulMMt(thetah[0], thetah[1], thetah[2], thetah[3], 10)
    momentsl  = getSimulMMt(thetal[0], thetal[1], thetal[2], thetal[3], 10)
    G[:,i]    = np.array((momentsh-momentsl)/(2*h[i]))
    print('{0} finished.'.format(i+1))

1 finished.
2 finished.
3 finished.
4 finished.


In [32]:
# Calculate lambda: Lambda=(G'*weight*G)\(G'*weight);
temp1  = np.matmul(np.matmul(G.T, w), G)
temp2  = np.matmul(G.T, w)
Lambda = np.matmul(np.linalg.inv(temp1), temp2)

In [33]:
# calculate the ste: ste=[(.^0.5)'];
temp1 = np.matmul(Lambda, var_matrix)
temp2 = (1+1/10)*np.matmul(temp1, Lambda.T)
ste = np.diag(temp2)**(0.5)
ste

array([0.08668218, 0.21146636, 0.32438855, 0.04739355])

In [34]:
# Conduct inference.
theta/ste

array([27.06623755, 94.82823221,  6.63373302,  6.75912946])

In [35]:
# Final parameters we have.
theta

array([ 2.34616035, 20.05298081,  2.15190703,  0.32033915])